In [ ]:
# load data
#input_file = "inputs/a_example.json"
input_file = "inputs/b_read_on.json"

import json
from collections import namedtuple

Library = namedtuple("Library", ["books", "speed", "cost"])

def to_library(o):
    return Library(o["books"], o["scanSpeed"], o["signupCost"])

o = json.load(open(input_file))
scores = o["scores"]
libraries = list(map(to_library, o["libraries"]))
L = len(libraries)
D = o["days"]
B = len(scores)

In [2]:
# Define constraints
from z3 import *

def z3_max(x,y):
    return If(x > y, x, y)

def z3_min(x,y):
    return If(x < y, x, y)

def z3_sum_of_bools(bools):
    """Sum of Bools"""
    return Sum([If(b, 1, 0) for b in bools])

s = Optimize()

scan_orders = IntVector("order", L)
for i, order in enumerate(scan_orders):
    s.add(order >= 0)
    s.add(order < L)
    for order_other in scan_orders[i+1:]:
        s.add(order != order_other)

books_sent = [[] for i in range(B)]
books_sent_by_lib = []
for i, order in enumerate(scan_orders):
    days_took_waiting = 0
    for j, order_other in enumerate(scan_orders):
        days_took_waiting += (order >= order_other) * libraries[j].cost
    lib = libraries[i]
    max_num_books_sent = z3_max(0, D - days_took_waiting) * lib.speed
    # books_sent = choose(libraries[i], librarnum_books_sent)
    lib_books_sent = BoolVector(f"sent_{i}", len(lib.books))
    books_sent_by_lib.append(lib_books_sent)
    s.add(z3_sum_of_bools(lib_books_sent) <= max_num_books_sent)
    for k, book in enumerate(lib.books):
        books_sent[book].append(lib_books_sent[k])

In [ ]:
# Define score function
score = 0
for i, list_sent in enumerate(books_sent):
    score += If(Or(list_sent), scores[i], 0)
score_obj = s.maximize(score)

In [ ]:
# Solve
if s.check() == sat:
    print("score:", score_obj.value())

In [ ]:
# Get results
model = s.model()
res_scan_orders = [model.eval(order).as_long() for order in scan_orders]
res_sent = [[bool(model.eval(b)) for b in books_sent_lib] for books_sent_lib in books_sent_by_lib]
res_signed_up = [any(rr) for rr in res_sent]

In [ ]:
# Inspect
print(res_scan_orders)
print(res_sent)
print(res_signed_up)

In [ ]:
# Output
import numpy as np
print(sum(res_signed_up))
for lib_id in np.argsort(res_scan_orders):
    lib = libraries[lib_id]
    res_sent_this = res_sent[lib_id]
    res_sent_sum = sum(res_sent_this)
    if res_sent_sum > 0:
        print(lib_id, res_sent_sum)
        books_scanned = []
        for b, res_sent_book in enumerate(res_sent_this):
            if res_sent_book:
                books_scanned.append(lib.books[b])
        print(*books_scanned)